In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install openai==0.28
!pip install pandas
!pip install tqdm
!pip install json5

In [ ]:
import openai
import pandas as pd
import json
from tqdm import tqdm

openai.api_key = "API-KEY"

def create_prompt(text):
    few_shot_examples = (
        "Here are examples of Early African American Vernacular English (EAAVE):\n"
        "1. Now, if yo' wants tuh put a fellah mind away, yo' kill a toadfrog an' tie a long string to 'im an' go tuh a swingin' limb in de woods, an' swing him tuh de sunrise side, an' every time de wind shake dat tree an' keep him a-swingin'\n"
        "2. Hit wuz only one ob us Marster's places cause he wuz one ob de richest en highest quality gentlemen in de whole country.\n"
        "3. Yo' take de man's socks an' a woman's sock, but chew gotta git dirty one whut he wear - git one of hern an' one of his'n, if dey done lives together.\n"
        f"\nHere is the input text: {text}\n"
        "Please rewrite the input text in Early African American Vernacular English."
    )
    return few_shot_examples

def translate_to_eaave(text):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a language model capable of translating text into Early African American Vernacular English (EAAVE)."},
                {"role": "user", "content": create_prompt(text)}
            ],
            max_tokens=500,
            temperature=0.7
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        print(f"Error translating text: {e}")
        return None

def process_csv(file_path, column_to_translate):
    df = pd.read_csv(file_path)
    translated_column = []
    total_rows = len(df)
    for idx, text in enumerate(tqdm(df[column_to_translate], desc=f"Translating {column_to_translate}"), start=1):
        translated_text = translate_to_eaave(text)
        translated_column.append(translated_text)
        print(f"Processed row {idx}/{total_rows}")
    df[f"EAAVE ({column_to_translate})"] = translated_column
    output_path = f"/content/drive/MyDrive/!!Multi-AAVENUE/Translated Datasets/{file_path.split('/')[-1].replace('.csv', '_EAAVE.csv')}"
    df.to_csv(output_path, index=False)
    print(f"Translated file saved to: {output_path}")

def process_json(file_path, key_to_translate):
    with open(file_path, "r") as file:
        data = json.load(file)
    translated_data = []
    total_items = len(data["propositional_logic"]["samples"])

    for idx, item in enumerate(tqdm(data["propositional_logic"]["samples"], desc=f"Translating {key_to_translate}"), start=1):
        item_copy = item.copy()
        item_copy[f"EAAVE ({key_to_translate})"] = translate_to_eaave(item[key_to_translate])
        translated_data.append(item_copy)
        print(f"Processed row {idx}/{total_items}")

    data["propositional_logic"]["samples"] = translated_data
    output_path = f"/content/drive/MyDrive/!!Multi-AAVENUE/Translated Datasets/{file_path.split('/')[-1].replace('.json', '_EAAVE.json')}"
    with open(output_path, "w") as file:
        json.dump(data, file, indent=4)
    print(f"Translated file saved to: {output_path}")

file_translation_map = {
    "/content/drive/MyDrive/!!Multi-AAVENUE/Datasets/FOLIO(1000).csv": "Premises",
    "/content/drive/MyDrive/!!Multi-AAVENUE/Datasets/GSM8K(1000).csv": "Original",
    "/content/drive/MyDrive/!!Multi-AAVENUE/Datasets/HumanEVAL(164).csv": "Prompt",
    "/content/drive/MyDrive/!!Multi-AAVENUE/Datasets/Logic Bench MCQ(480).json": "context",
    "/content/drive/MyDrive/!!Multi-AAVENUE/Datasets/Logic Bench YN(500).json": "context",
    "/content/drive/MyDrive/!!Multi-AAVENUE/Datasets/MBPP(374).csv": "Original",
    "/content/drive/MyDrive/!!Multi-AAVENUE/Datasets/SVAMP(700).csv": "Original"
}

for file_path, column_or_key in file_translation_map.items():
    if file_path.endswith(".csv"):
        process_csv(file_path, column_or_key)
    elif file_path.endswith(".json"):
        process_json(file_path, column_or_key)

print("All translations completed and saved to /content/drive/MyDrive/!!Multi-AAVENUE/Translated Datasets.")